In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import glob
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu" 

pytorch version: 1.9.0+cu102
GPU 사용 가능 여부: True


In [ ]:
df = pd.read_csv('./drive/MyDrive/포토 뮤직리스트/image_labels/emo_labeled.csv', converters={'emotion':eval}).iloc[:, 1:]

In [ ]:
gray = []

for i in range(len(df)):
  try:
    if c(Image.open(df['fig path'][i])).size()[0] != 3:
      gray.append(i)
  except:
    print(i)
    gray.append(i)

In [ ]:
print(gray)

[596, 2099, 3565, 3685, 3910, 4262, 5845, 6417]


In [ ]:
gray

[]

In [ ]:
df = df.drop(gray)

In [ ]:
df.to_csv('./drive/MyDrive/포토 뮤직리스트/image_labels/emo_labeled_color.csv')

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=23)

In [ ]:
train

,fig path,emotion
6802,/content/drive/MyDrive/포토 뮤직리스트/images...,"[0, 0, 1, 0, 1]"
4719,/content/drive/MyDrive/포토 뮤직리스트/love/u...,"[0, 0, 1, 1, 0]"
578,/content/drive/MyDrive/포토 뮤직리스트/images...,"[0, 0, 0, 1, 0]"
1188,/content/drive/MyDrive/포토 뮤직리스트/images...,"[0, 1, 0, 1, 0]"
4669,/content/drive/MyDrive/포토 뮤직리스트/love/u...,"[1, 0, 0, 0, 0]"
...,...,...
39,/content/drive/MyDrive/포토 뮤직리스트/images...,"[0, 1, 0, 0, 0]"
347,/content/drive/MyDrive/포토 뮤직리스트/images...,"[0, 0, 0, 0, 1]"
6182,/content/drive/MyDrive/포토 뮤직리스트/images/1_resiz...,"[0, 1, 0, 0, 0]"
9264,/content/drive/MyDrive/포토 뮤직리스트/images...,"[0, 1, 0, 0, 0]"


In [ ]:
class EmoImageDataset(Dataset):
    def __init__(self, mode, transform=None):
        if mode == 'train':
          self.all_data = list(train['fig path'].values)
          self.labels = list(train['emotion'].values)
        else:
          self.all_data = list(test['fig path'].values)
          self.labels = list(test['emotion'].values)
        self.transform = transform
    
    def __getitem__(self, index):
        data_path = self.all_data[index]
        img = Image.open(data_path)
        if self.transform != None:
          img = self.transform(img)
        
        label = np.array(self.labels[index])
        return [img, label]
    
    def __len__(self):
        length = len(self.all_data)
        return length
        

In [ ]:
learning_rate = 0.001
batch_size = 32
num_epochs = 10

In [ ]:
transform =  transforms.Compose([
        transforms.Resize([256, 256]),
        transforms.ToTensor(),
    ])

train_data = EmoImageDataset(mode='train', transform=transform)
test_data = EmoImageDataset(mode='test', transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
class Resnext50(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet = models.resnext50_32x4d(pretrained=True)
        resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        return self.sigm(self.base_model(x))


model = Resnext50(5)

In [ ]:
criterion = nn.BCELoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

model = model.to(device).double()
val_every = 1
saved_dir = './drive/MyDrive/포토 뮤직리스트/Modelling/'

In [72]:
temp1 = torch.Tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [1., 1., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 1., 0.],
        [1., 1., 0., 1., 0.],
        [1., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 1., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]])
temp2 = torch.Tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [76]:
temp1 == temp2

tensor([[False, False,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True, False,  True, False,  True],
        [ True,  True,  True,  True,  True],
        [False,  True,  True, False,  True],
        [ True, False,  True, False,  True],
        [ True, False,  True, False, False],
        [ True,  True,  True,  True,  True],
        [ True, False,  True, False,  True],
        [False, False,  True,  True,  True],
        [ True, False, False, False, False],
        [ True,  True,  True, False,  True],
        [ True,  True,  True, False,  True],
        [ True,  True,  True, False,  True],
        [ True, False,  True, False,  True],
        [ True,  True,  True,  True,  True],
        [ True, False,  True, False,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True],
        [False, False,  True,  True,  True],
        [False, False,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ 

In [79]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device).double(), labels.to(device).double()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total += imgs.size(0) * 5
            argmax = torch.round(outputs)
            temp1 = labels
            temp2 = argmax
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Validation #{}  Accuracy: {:.2f}%  Average Loss: {:.4f}'.format(epoch, correct / total * 100, avrg_loss))
    model.train()
    return avrg_loss

def save_model(model, saved_dir, file_name='best_model.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point, output_path)

In [ ]:
print('Start training..')
best_loss = 9999999
for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device).double(), labels.to(device).double()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()
        argmax = torch.round(outputs)
        accuracy = (labels == argmax).float().mean()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, num_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))

    if (epoch + 1) % val_every == 0:
        avrg_loss = validation(epoch + 1, model, test_loader, criterion, device)
        if avrg_loss < best_loss:
            print('Best performance at epoch: {}'.format(epoch + 1))
            print('Save model in', saved_dir)
            best_loss = avrg_loss
            save_model(model, saved_dir)

Start training..


In [ ]:
for i, (data, labels) in enumerate(train_loader):
  print(i)
  print(labels)

0
[tensor([0, 0, 0, 1, 0, 0, 1, 0, 0, 0]), tensor([1, 0, 1, 1, 1, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), tensor([0, 1, 1, 0, 0, 0, 1, 1, 1, 1]), tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])]
1
[tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 0, 1, 0, 0, 1, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0]), tensor([0, 0, 1, 0, 0, 1, 1, 1, 1, 1]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


KeyboardInterrupt: ignored